In [1]:
import csv,sys
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, train_test_split 
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [3]:
f_train_data = open('/content/assignment1_training_data.txt','r')
#print(f)
reader_train_data = list(csv.reader(f_train_data,delimiter='\t'))

data_train=[];
for item in reader_train_data:
    #labels.append(item[0])    
    data_train.append(item[0])

f_train_label = open('/content/assignment1_training_data_labels.txt','r')
#print(f)
reader_train_label = list(csv.reader(f_train_label,delimiter='\t'))

labels_train=[];
for item in reader_train_label:
    #labels.append(item[0])    
    labels_train.append(item[0])

f_test_data = open('/content/assignment1_test_data.txt','r')
#print(f)
reader_test_data = list(csv.reader(f_test_data,delimiter='\t'))

data_test=[];
for item in reader_test_data:
    #labels.append(item[0])    
    data_test.append(item[0])  

analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
  return (PorterStemmer().stem(w) for w in analyzer(doc))

def lemmatized_words(doc):
  return (WordNetLemmatizer().lemmatize(w) for w in analyzer(doc))


In [ ]:
# print(analyzer(data_train[0]))
# t= lemmatized_words(data_train[0])
# print(t[0])
# print(data_train[0])

In [4]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text, type):
    """
        text: a string
        
        return: modified initial string
    """
    # text = BeautifulSoup(text, "lxml").text # HTML decoding
    # text = text.lower() # lowercase text
    # text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    # text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    #text = ' '.join(word for word in analyzer(text) if word not in STOPWORDS) # delete stopwors from text
    #text = ' '.join(WordNetLemmatizer().lemmatize(w) for w in analyzer(text))
    if type == 1:
      text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    elif type == 2:
      #print('only_stop')
      text = ' '.join(word for word in analyzer(text) if word not in STOPWORDS)
    
    elif type == 3:
      text = ' '.join(PorterStemmer().stem(w) for w in analyzer(text))
    elif type == 4:
      text = ' '.join(WordNetLemmatizer().lemmatize(w) for w in analyzer(text))
    elif type == 5:
      text = ' '.join(word for word in analyzer(text) if word not in STOPWORDS)
      text = ' '.join(PorterStemmer().stem(w) for w in analyzer(text))
    elif type == 6:
      text = ' '.join(word for word in analyzer(text) if word not in STOPWORDS)
      text = ' '.join(WordNetLemmatizer().lemmatize(w) for w in analyzer(text))
    return text

df_train = pd.DataFrame(data_train)
# print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())
# df_train[0] = df_train[0].apply(clean_text)
# print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())
#df_train[0]    

In [ ]:
# df_train = pd.DataFrame(data_train)
# print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())
# df_train[0] = df_train[0].apply(clean_text)
# print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())
# #df_train[0]

In [ ]:
# df_test = pd.DataFrame(data_test)
# print('words_per before',df_test[0].apply(lambda x: len(x.split(' '))).sum())
# df_test[0] = df_test[0].apply(clean_text)
# print('words_per after',df_test[0].apply(lambda x: len(x.split(' '))).sum())
# df_test[0]

In [ ]:
data_test
#labels_train

In [5]:
import pandas as pd
new_row = {'model':['demo'],'precision': [0],'recall':[0], 'micro_f1': [0],'macro_f1':[0] }
#new_row = {'model':['demo'],'preprocessing_used':['demo'],'precision': [0],'recall':[0], 'micro_f1': [0],'macro_f1':[0] }
df0= pd.DataFrame(new_row) 
df0

,model,precision,recall,micro_f1,macro_f1
0,demo,0,0,0,0


In [24]:
def evaluation(trn_data,trn_cat,tst_cat,tst_data,predicted,model,analyser, df0):
    #   def evaluation(tst_cat,predicted,model,analyser, df0):
    # print('\n Total documents in the training set: '+str(len(trn_data))+'\n')    
    # print('\n Total documents in the test set: '+str(len(tst_data))+'\n')
    print ('\n Confusion Matrix \n')  
    print (confusion_matrix(tst_cat, predicted))  

    pr=precision_score(tst_cat, predicted, average='macro') 
    print ('\n Precision:'+str(pr)) 

    rl=recall_score(tst_cat, predicted, average='macro') 
    print ('\n Recall:'+str(rl))

    fm1=f1_score(tst_cat, predicted, average='macro') 
    print ('\n Macro Averaged F1-Score :'+str(fm1))

    fm2=f1_score(tst_cat, predicted, average='micro') 
    print ('\n Mircro Averaged F1-Score:'+str(fm2))
    new_row = {'model':model,'preprocessing_used':analyser,'precision': pr,'recall':rl, 'micro_f1': fm1,'macro_f1':fm2 }
    #new_row = {'model':model,'precision': pr,'recall':rl, 'micro_f1': fm1,'macro_f1':fm2 }
    df0= df0.append(new_row,ignore_index=True) 
    print(df0)
    return df0


#to compare best preprocessing technique

In [7]:
import pandas as pd
df = pd.read_csv('/content/assignment1_test_data_labels.txt', header =None)
df

,0
0,positive
1,neutral
2,negative
3,neutral
4,positive
...,...
448,negative
449,neutral
450,neutral
451,positive


In [ ]:
df1 = pd.read_csv('/content/ananya_singha_18032_test_labels_csv.csv')
df1= df1.drop(columns= ['Unnamed: 0'] , axis =0)
df1

In [ ]:
evaluation(df,df1,'hh',df0)


 Confusion Matrix 

[[ 52   0   9]
 [  3 264  11]
 [  8  16  90]]

 Precision:0.8621452621452622

 Recall:0.863857662791251

 Macro Averaged F1-Score :0.8628392217101895

 Mircro Averaged F1-Score:0.8962472406181016
  model  precision    recall  micro_f1  macro_f1
0  demo   0.000000  0.000000  0.000000  0.000000
1    hh   0.862145  0.863858  0.862839  0.896247


,model,precision,recall,micro_f1,macro_f1
0,demo,0.000000,0.000000,0.000000,0.000000
1,hh,0.862145,0.863858,0.862839,0.896247


# word to vec

1

In [9]:
!wget -c "https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g"

--2022-09-16 22:46:55--  https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g
Resolving drive.google.com (drive.google.com)... 142.250.145.101, 142.250.145.113, 142.250.145.102, ...
Connecting to drive.google.com (drive.google.com)|142.250.145.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘edit?resourcekey=0-wjGZdNAUop6WykTtMip30g’

edit?resourcekey=0-     [ <=>                ]  66.58K  --.-KB/s    in 0.09s   

2022-09-16 22:46:57 (746 KB/s) - ‘edit?resourcekey=0-wjGZdNAUop6WykTtMip30g’ saved [68175]



In [23]:

#%%time
from gensim.models import Word2Vec

wv = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/MyDrive/basic_mnist/GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)


def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])


def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens    

#no pre

In [27]:
df_train = pd.DataFrame(data_train)
print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())
df_test = pd.DataFrame(data_test)
df_train[0] = df_train[0].apply(clean_text, args= (1,))
print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())

train_x, test_x, train_y, test_y = train_test_split(df_train, labels_train, test_size=0.2, random_state = 42)

test_tokenized = test_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values
train_tokenized = train_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values


X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

clf=LogisticRegression(class_weight='balanced', solver='newton-cg')
clf.fit(X_train_word_average,train_y)
predicted = clf.predict(X_test_word_average)
predicted =list(predicted)
#evaluation(train_x,train_y,test_y,test_x,predicted,'no_preprocessing',df)
df0 = evaluation(X_train_word_average,train_y,test_y,X_test_word_average,predicted,'Word2vec','no_preprocessing',df0)

words_per before 40627
words_per after 29410


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  from ipykernel import kernelapp as app



 Confusion Matrix 

[[ 38   2  12]
 [  8 201  10]
 [ 21  10  61]]

 Precision:0.7485886366572027

 Recall:0.7705403094027274

 Macro Averaged F1-Score :0.7554512916277623

 Mircro Averaged F1-Score:0.8264462809917356
      model  precision    recall  micro_f1  macro_f1  \
0      demo   0.000000  0.000000  0.000000  0.000000   
1  Word2vec   0.748589  0.770540  0.755451  0.826446   
2  Word2vec   0.722650  0.750688  0.733101  0.804408   
3  Word2vec   0.729422  0.743227  0.730539  0.809917   
4  Word2vec   0.753520  0.768182  0.756126  0.820937   
5  Word2vec   0.694979  0.723547  0.702064  0.776860   
6  Word2vec   0.716415  0.734502  0.722823  0.801653   
7  Word2vec   0.722650  0.750688  0.733101  0.804408   
8  Word2vec   0.748589  0.770540  0.755451  0.826446   

           preprocessing_used  
0                         NaN  
1            no_preprocessing  
2              only_stopwords  
3          only_stemmed_words  
4       only_lemmatized_words  
5    stopword + stemmed_words

# only_stop

In [28]:
df_train = pd.DataFrame(data_train)
print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())

df_train[0] = df_train[0].apply(clean_text, args= (2,))
print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())

train_x, test_x, train_y, test_y = train_test_split(df_train, labels_train, test_size=0.2, random_state = 42)

test_tokenized = test_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values
train_tokenized = train_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values


X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

clf=LogisticRegression(class_weight='balanced', solver='newton-cg')
clf.fit(X_train_word_average,train_y)
predicted = clf.predict(X_test_word_average)
predicted =list(predicted)
#evaluation(train_x,train_y,test_y,test_x,predicted,'no_preprocessing',df)
df0 = evaluation(X_train_word_average,train_y,test_y,X_test_word_average,predicted,'Word2vec','only_stopwords',df0)

words_per before 40627
words_per after 23360


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  from ipykernel import kernelapp as app



 Confusion Matrix 

[[ 36   1  15]
 [ 12 194  13]
 [ 19  11  62]]

 Precision:0.7226499648467479

 Recall:0.7506884948814668

 Macro Averaged F1-Score :0.7331006248653308

 Mircro Averaged F1-Score:0.8044077134986226
      model  precision    recall  micro_f1  macro_f1  \
0      demo   0.000000  0.000000  0.000000  0.000000   
1  Word2vec   0.748589  0.770540  0.755451  0.826446   
2  Word2vec   0.722650  0.750688  0.733101  0.804408   
3  Word2vec   0.729422  0.743227  0.730539  0.809917   
4  Word2vec   0.753520  0.768182  0.756126  0.820937   
5  Word2vec   0.694979  0.723547  0.702064  0.776860   
6  Word2vec   0.716415  0.734502  0.722823  0.801653   
7  Word2vec   0.722650  0.750688  0.733101  0.804408   
8  Word2vec   0.748589  0.770540  0.755451  0.826446   
9  Word2vec   0.722650  0.750688  0.733101  0.804408   

           preprocessing_used  
0                         NaN  
1            no_preprocessing  
2              only_stopwords  
3          only_stemmed_words  
4    

/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


# only stem

In [29]:
df_train = pd.DataFrame(data_train)
print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())

df_train[0] = df_train[0].apply(clean_text,args= (3,))
print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())

train_x, test_x, train_y, test_y = train_test_split(df_train, labels_train, test_size=0.2, random_state = 42)

test_tokenized = test_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values
train_tokenized = train_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values


X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

clf=LogisticRegression(class_weight='balanced', solver='newton-cg')
clf.fit(X_train_word_average,train_y)
predicted = clf.predict(X_test_word_average)
predicted =list(predicted)
#evaluation(train_x,train_y,test_y,test_x,predicted,'no_preprocessing',df)
df0 = evaluation(X_train_word_average,train_y,test_y,X_test_word_average,predicted,'Word2vec','only_stemmed_words',df0)

words_per before 40627
words_per after 34802


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  from ipykernel import kernelapp as app



 Confusion Matrix 

[[ 36   5  11]
 [  9 201   9]
 [ 24  11  57]]

 Precision:0.7294220506001712

 Recall:0.7432270429590262

 Macro Averaged F1-Score :0.7305386279852181

 Mircro Averaged F1-Score:0.8099173553719008
       model  precision    recall  micro_f1  macro_f1  \
0       demo   0.000000  0.000000  0.000000  0.000000   
1   Word2vec   0.748589  0.770540  0.755451  0.826446   
2   Word2vec   0.722650  0.750688  0.733101  0.804408   
3   Word2vec   0.729422  0.743227  0.730539  0.809917   
4   Word2vec   0.753520  0.768182  0.756126  0.820937   
5   Word2vec   0.694979  0.723547  0.702064  0.776860   
6   Word2vec   0.716415  0.734502  0.722823  0.801653   
7   Word2vec   0.722650  0.750688  0.733101  0.804408   
8   Word2vec   0.748589  0.770540  0.755451  0.826446   
9   Word2vec   0.722650  0.750688  0.733101  0.804408   
10  Word2vec   0.729422  0.743227  0.730539  0.809917   

            preprocessing_used  
0                          NaN  
1             no_preprocessing 

#only lem

In [32]:
df_train = pd.DataFrame(data_train)
print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())

df_train[0] = df_train[0].apply(clean_text, args= (4,))
print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())

train_x, test_x, train_y, test_y = train_test_split(df_train, labels_train, test_size=0.2, random_state = 42)

test_tokenized = test_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values
train_tokenized = train_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values


X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

clf=LogisticRegression(class_weight='balanced', solver='newton-cg')
clf.fit(X_train_word_average,train_y)
predicted = clf.predict(X_test_word_average)
predicted =list(predicted)
#evaluation(train_x,train_y,test_y,test_x,predicted,'no_preprocessing',df)
df0 = evaluation(X_train_word_average,train_y,test_y,X_test_word_average,predicted,'Word2vec','only_lemmatized_words',df0)

words_per before 40627
words_per after 34802


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  from ipykernel import kernelapp as app



 Confusion Matrix 

[[ 39   4   9]
 [  9 200  10]
 [ 18  15  59]]

 Precision:0.753520452150589

 Recall:0.768182118986169

 Macro Averaged F1-Score :0.7561255351145952

 Mircro Averaged F1-Score:0.8209366391184573
       model  precision    recall  micro_f1  macro_f1  \
0       demo   0.000000  0.000000  0.000000  0.000000   
1   Word2vec   0.748589  0.770540  0.755451  0.826446   
2   Word2vec   0.722650  0.750688  0.733101  0.804408   
3   Word2vec   0.729422  0.743227  0.730539  0.809917   
4   Word2vec   0.753520  0.768182  0.756126  0.820937   
5   Word2vec   0.694979  0.723547  0.702064  0.776860   
6   Word2vec   0.716415  0.734502  0.722823  0.801653   
7   Word2vec   0.722650  0.750688  0.733101  0.804408   
8   Word2vec   0.748589  0.770540  0.755451  0.826446   
9   Word2vec   0.722650  0.750688  0.733101  0.804408   
10  Word2vec   0.729422  0.743227  0.730539  0.809917   
11  Word2vec   0.753520  0.768182  0.756126  0.820937   

            preprocessing_used  
0        

#bstem

In [33]:
df_train = pd.DataFrame(data_train)
print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())

df_train[0] = df_train[0].apply(clean_text,args= (5,))
print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())

train_x, test_x, train_y, test_y = train_test_split(df_train, labels_train, test_size=0.2, random_state = 42)

test_tokenized = test_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values
train_tokenized = train_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values


X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

clf=LogisticRegression(class_weight='balanced', solver='newton-cg')
clf.fit(X_train_word_average,train_y)
predicted = clf.predict(X_test_word_average)
predicted =list(predicted)
#evaluation(train_x,train_y,test_y,test_x,predicted,'no_preprocessing',df)
df0 = evaluation(X_train_word_average,train_y,test_y,X_test_word_average,predicted,'Word2vec','stopword + stemmed_words',df0)

words_per before 40627
words_per after 23360


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  from ipykernel import kernelapp as app



 Confusion Matrix 

[[ 37   3  12]
 [ 13 191  15]
 [ 24  14  54]]

 Precision:0.6949786324786325

 Recall:0.7235470339996843

 Macro Averaged F1-Score :0.7020642090290532

 Mircro Averaged F1-Score:0.7768595041322314
       model  precision    recall  micro_f1  macro_f1  \
0       demo   0.000000  0.000000  0.000000  0.000000   
1   Word2vec   0.748589  0.770540  0.755451  0.826446   
2   Word2vec   0.722650  0.750688  0.733101  0.804408   
3   Word2vec   0.729422  0.743227  0.730539  0.809917   
4   Word2vec   0.753520  0.768182  0.756126  0.820937   
5   Word2vec   0.694979  0.723547  0.702064  0.776860   
6   Word2vec   0.716415  0.734502  0.722823  0.801653   
7   Word2vec   0.722650  0.750688  0.733101  0.804408   
8   Word2vec   0.748589  0.770540  0.755451  0.826446   
9   Word2vec   0.722650  0.750688  0.733101  0.804408   
10  Word2vec   0.729422  0.743227  0.730539  0.809917   
11  Word2vec   0.753520  0.768182  0.756126  0.820937   
12  Word2vec   0.694979  0.723547  0.7020

#blem

In [34]:
df_train = pd.DataFrame(data_train)
print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())

df_train[0] = df_train[0].apply(clean_text, args= (6,))
print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())

train_x, test_x, train_y, test_y = train_test_split(df_train, labels_train, test_size=0.2, random_state = 42)

test_tokenized = test_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values
train_tokenized = train_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values


X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

clf=LogisticRegression(class_weight='balanced', solver='newton-cg')
clf.fit(X_train_word_average,train_y)
predicted = clf.predict(X_test_word_average)
predicted =list(predicted)
#evaluation(train_x,train_y,test_y,test_x,predicted,'no_preprocessing',df)
df0 = evaluation(X_train_word_average,train_y,test_y,X_test_word_average,predicted,'Word2vec','stopword +lemmatized_words',df0)

words_per before 40627
words_per after 23360


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  from ipykernel import kernelapp as app



 Confusion Matrix 

[[ 33   4  15]
 [ 13 196  10]
 [ 19  11  62]]

 Precision:0.7164153128199541

 Recall:0.7345018656811391

 Macro Averaged F1-Score :0.7228226337489687

 Mircro Averaged F1-Score:0.8016528925619834
       model  precision    recall  micro_f1  macro_f1  \
0       demo   0.000000  0.000000  0.000000  0.000000   
1   Word2vec   0.748589  0.770540  0.755451  0.826446   
2   Word2vec   0.722650  0.750688  0.733101  0.804408   
3   Word2vec   0.729422  0.743227  0.730539  0.809917   
4   Word2vec   0.753520  0.768182  0.756126  0.820937   
5   Word2vec   0.694979  0.723547  0.702064  0.776860   
6   Word2vec   0.716415  0.734502  0.722823  0.801653   
7   Word2vec   0.722650  0.750688  0.733101  0.804408   
8   Word2vec   0.748589  0.770540  0.755451  0.826446   
9   Word2vec   0.722650  0.750688  0.733101  0.804408   
10  Word2vec   0.729422  0.743227  0.730539  0.809917   
11  Word2vec   0.753520  0.768182  0.756126  0.820937   
12  Word2vec   0.694979  0.723547  0.7020

# Bag of words

 # 0. nothing

In [21]:
len(data_train),len(labels_train)

(1811, 1811)

In [35]:
#clf=MultinomialNB(alpha=0,fit_prior=True, class_prior=None)
clf=LogisticRegression(class_weight='balanced', solver='newton-cg')  
vectorizer=TfidfVectorizer(ngram_range=(2,3),token_pattern=r'\b\w+\b')
tfidf = vectorizer.fit_transform(data_train)
tfidf = tfidf.toarray()

# Training and Test Split           
  
train_x, test_x, train_y, test_y = train_test_split(tfidf, labels_train, test_size=0.20, random_state=42,stratify=labels_train)

#Classificaion    
clf.fit(train_x,train_y)
predicted = clf.predict(test_x)
predicted =list(predicted)
#evaluation(train_x,train_y,test_y,test_x,predicted,'no_preprocessing',df)
df0 = evaluation(train_x,train_y,test_y,test_x,predicted,'Bag of words','no_preprocessing',df0)


 Confusion Matrix 

[[ 23  14  12]
 [  0 222   1]
 [  3  48  40]]

 Precision:0.8070075021975102

 Recall:0.6348212965765817

 Macro Averaged F1-Score :0.6815428446197677

 Mircro Averaged F1-Score:0.7851239669421488
           model  precision    recall  micro_f1  macro_f1  \
0           demo   0.000000  0.000000  0.000000  0.000000   
1       Word2vec   0.748589  0.770540  0.755451  0.826446   
2       Word2vec   0.722650  0.750688  0.733101  0.804408   
3       Word2vec   0.729422  0.743227  0.730539  0.809917   
4       Word2vec   0.753520  0.768182  0.756126  0.820937   
5       Word2vec   0.694979  0.723547  0.702064  0.776860   
6       Word2vec   0.716415  0.734502  0.722823  0.801653   
7       Word2vec   0.722650  0.750688  0.733101  0.804408   
8       Word2vec   0.748589  0.770540  0.755451  0.826446   
9       Word2vec   0.722650  0.750688  0.733101  0.804408   
10      Word2vec   0.729422  0.743227  0.730539  0.809917   
11      Word2vec   0.753520  0.768182  0.756126  0

In [ ]:
tfidf.shape

(1811, 47662)

In [ ]:
predicted

#1. only stop word

In [36]:
#clf=MultinomialNB(alpha=0,fit_prior=True, class_prior=None) 
clf=LogisticRegression(class_weight='balanced', solver='newton-cg')  
vectorizer=TfidfVectorizer(stop_words='english',ngram_range=(2,3),token_pattern=r'\b\w+\b')
tfidf = vectorizer.fit_transform(data_train)
tfidf = tfidf.toarray()
# Training and Test Split           
  
train_x, test_x, train_y, test_y = train_test_split(tfidf, labels_train, test_size=0.20, random_state=42,stratify=labels_train)

#Classificaion    
clf.fit(train_x,train_y)
predicted = clf.predict(test_x)
predicted =list(predicted)
df0 =evaluation(train_x,train_y,test_y,test_x,predicted,'Bag of words','only_stopword',df0)


 Confusion Matrix 

[[ 18  21  10]
 [  0 220   3]
 [  6  52  33]]

 Precision:0.7394148488895484

 Recall:0.5721771288715555

 Macro Averaged F1-Score :0.6092052293478663

 Mircro Averaged F1-Score:0.7465564738292011
           model  precision    recall  micro_f1  macro_f1  \
0           demo   0.000000  0.000000  0.000000  0.000000   
1       Word2vec   0.748589  0.770540  0.755451  0.826446   
2       Word2vec   0.722650  0.750688  0.733101  0.804408   
3       Word2vec   0.729422  0.743227  0.730539  0.809917   
4       Word2vec   0.753520  0.768182  0.756126  0.820937   
5       Word2vec   0.694979  0.723547  0.702064  0.776860   
6       Word2vec   0.716415  0.734502  0.722823  0.801653   
7       Word2vec   0.722650  0.750688  0.733101  0.804408   
8       Word2vec   0.748589  0.770540  0.755451  0.826446   
9       Word2vec   0.722650  0.750688  0.733101  0.804408   
10      Word2vec   0.729422  0.743227  0.730539  0.809917   
11      Word2vec   0.753520  0.768182  0.756126  0

#. only stemmed_words

In [37]:
#clf=MultinomialNB(alpha=0,fit_prior=True, class_prior=None) 
clf=LogisticRegression(class_weight='balanced', solver='newton-cg')   
vectorizer=TfidfVectorizer(analyzer = stemmed_words,ngram_range=(2,3),token_pattern=r'\b\w+\b')
tfidf = vectorizer.fit_transform(data_train)
tfidf = tfidf.toarray()
# Training and Test Split           
  
train_x, test_x, train_y, test_y = train_test_split(tfidf, labels_train, test_size=0.20, random_state=42,stratify=labels_train)

#Classificaion    
clf.fit(train_x,train_y)
predicted = clf.predict(test_x)
predicted =list(predicted)
df0 = evaluation(train_x,train_y,test_y,test_x,predicted,'Bag of words', 'only_stemmed_words',df0)


 Confusion Matrix 

[[ 38   6   5]
 [  3 213   7]
 [  6  13  72]]

 Precision:0.8612523145721972

 Recall:0.8406253153210231

 Macro Averaged F1-Score :0.8502625152625152

 Mircro Averaged F1-Score:0.8898071625344353
           model  precision    recall  micro_f1  macro_f1  \
0           demo   0.000000  0.000000  0.000000  0.000000   
1       Word2vec   0.748589  0.770540  0.755451  0.826446   
2       Word2vec   0.722650  0.750688  0.733101  0.804408   
3       Word2vec   0.729422  0.743227  0.730539  0.809917   
4       Word2vec   0.753520  0.768182  0.756126  0.820937   
5       Word2vec   0.694979  0.723547  0.702064  0.776860   
6       Word2vec   0.716415  0.734502  0.722823  0.801653   
7       Word2vec   0.722650  0.750688  0.733101  0.804408   
8       Word2vec   0.748589  0.770540  0.755451  0.826446   
9       Word2vec   0.722650  0.750688  0.733101  0.804408   
10      Word2vec   0.729422  0.743227  0.730539  0.809917   
11      Word2vec   0.753520  0.768182  0.756126  0

# only lemmatized_words

In [38]:
#clf=MultinomialNB(alpha=0,fit_prior=True, class_prior=None) 
clf=LogisticRegression(class_weight='balanced', solver='newton-cg')  
vectorizer=TfidfVectorizer(analyzer = lemmatized_words,ngram_range=(2,3),token_pattern=r'\b\w+\b')
tfidf = vectorizer.fit_transform(data_train)
tfidf = tfidf.toarray()
# Training and Test Split           
  
train_x, test_x, train_y, test_y = train_test_split(tfidf, labels_train, test_size=0.20, random_state=42,stratify=labels_train)

#Classificaion    
clf.fit(train_x,train_y)
predicted = clf.predict(test_x)
predicted =list(predicted)
df0= evaluation(train_x,train_y,test_y,test_x,predicted,'Bag of words', 'only_lemmatized_words',df0)


 Confusion Matrix 

[[ 37   6   6]
 [  4 214   5]
 [  5  14  72]]

 Precision:0.8621158733783144

 Recall:0.8353173625434995

 Macro Averaged F1-Score :0.8476920816339497

 Mircro Averaged F1-Score:0.8898071625344353
           model  precision    recall  micro_f1  macro_f1  \
0           demo   0.000000  0.000000  0.000000  0.000000   
1       Word2vec   0.748589  0.770540  0.755451  0.826446   
2       Word2vec   0.722650  0.750688  0.733101  0.804408   
3       Word2vec   0.729422  0.743227  0.730539  0.809917   
4       Word2vec   0.753520  0.768182  0.756126  0.820937   
5       Word2vec   0.694979  0.723547  0.702064  0.776860   
6       Word2vec   0.716415  0.734502  0.722823  0.801653   
7       Word2vec   0.722650  0.750688  0.733101  0.804408   
8       Word2vec   0.748589  0.770540  0.755451  0.826446   
9       Word2vec   0.722650  0.750688  0.733101  0.804408   
10      Word2vec   0.729422  0.743227  0.730539  0.809917   
11      Word2vec   0.753520  0.768182  0.756126  0

# stopword + stemmed_words

In [39]:
#clf=MultinomialNB(alpha=0,fit_prior=True, class_prior=None)
clf=LogisticRegression(class_weight='balanced', solver='newton-cg')   
vectorizer=TfidfVectorizer(stop_words='english',analyzer = stemmed_words, ngram_range=(2,3),token_pattern=r'\b\w+\b')
tfidf = vectorizer.fit_transform(data_train)
tfidf = tfidf.toarray()
# Training and Test Split           
  
train_x, test_x, train_y, test_y = train_test_split(tfidf, labels_train, test_size=0.20, random_state=42,stratify=labels_train)

#Classificaion    
clf.fit(train_x,train_y)
predicted = clf.predict(test_x)
predicted =list(predicted)
df0 = evaluation(train_x,train_y,test_y,test_x,predicted, 'Bag of words','stopword + stemmed_words', df0)


 Confusion Matrix 

[[ 38   6   5]
 [  3 213   7]
 [  6  13  72]]

 Precision:0.8612523145721972

 Recall:0.8406253153210231

 Macro Averaged F1-Score :0.8502625152625152

 Mircro Averaged F1-Score:0.8898071625344353
           model  precision    recall  micro_f1  macro_f1  \
0           demo   0.000000  0.000000  0.000000  0.000000   
1       Word2vec   0.748589  0.770540  0.755451  0.826446   
2       Word2vec   0.722650  0.750688  0.733101  0.804408   
3       Word2vec   0.729422  0.743227  0.730539  0.809917   
4       Word2vec   0.753520  0.768182  0.756126  0.820937   
5       Word2vec   0.694979  0.723547  0.702064  0.776860   
6       Word2vec   0.716415  0.734502  0.722823  0.801653   
7       Word2vec   0.722650  0.750688  0.733101  0.804408   
8       Word2vec   0.748589  0.770540  0.755451  0.826446   
9       Word2vec   0.722650  0.750688  0.733101  0.804408   
10      Word2vec   0.729422  0.743227  0.730539  0.809917   
11      Word2vec   0.753520  0.768182  0.756126  0

# stopword +lemmatized_words

In [40]:
#clf=MultinomialNB(alpha=0,fit_prior=True, class_prior=None)  
clf=LogisticRegression(class_weight='balanced', solver='newton-cg') 
vectorizer=TfidfVectorizer(stop_words='english',analyzer = lemmatized_words,ngram_range=(2,3),token_pattern=r'\b\w+\b')
tfidf = vectorizer.fit_transform(data_train)
tfidf = tfidf.toarray()
# Training and Test Split           
  
train_x, test_x, train_y, test_y = train_test_split(tfidf, labels_train, test_size=0.20, random_state=42,stratify=labels_train)

#Classificaion    
clf.fit(train_x,train_y)
predicted = clf.predict(test_x)
predicted =list(predicted)
df0 =evaluation(train_x,train_y,test_y,test_x,predicted,'Bag of words', 'stopword +lemmatized_words', df0)


 Confusion Matrix 

[[ 37   6   6]
 [  4 214   5]
 [  5  14  72]]

 Precision:0.8621158733783144

 Recall:0.8353173625434995

 Macro Averaged F1-Score :0.8476920816339497

 Mircro Averaged F1-Score:0.8898071625344353
           model  precision    recall  micro_f1  macro_f1  \
0           demo   0.000000  0.000000  0.000000  0.000000   
1       Word2vec   0.748589  0.770540  0.755451  0.826446   
2       Word2vec   0.722650  0.750688  0.733101  0.804408   
3       Word2vec   0.729422  0.743227  0.730539  0.809917   
4       Word2vec   0.753520  0.768182  0.756126  0.820937   
5       Word2vec   0.694979  0.723547  0.702064  0.776860   
6       Word2vec   0.716415  0.734502  0.722823  0.801653   
7       Word2vec   0.722650  0.750688  0.733101  0.804408   
8       Word2vec   0.748589  0.770540  0.755451  0.826446   
9       Word2vec   0.722650  0.750688  0.733101  0.804408   
10      Word2vec   0.729422  0.743227  0.730539  0.809917   
11      Word2vec   0.753520  0.768182  0.756126  0

In [ ]:
dfoo = df0.drop(0)
dfoo = dfoo.drop(13)
dfoo

,model,preprocessing_used,precision,recall,micro_f1,macro_f1
1,Word2vec,no_preprocessing,0.826446,0.826446,0.755451,0.826446
2,Word2vec,only_stopwords,0.804408,0.804408,0.733101,0.804408
3,Word2vec,only_stemmed_words,0.809917,0.809917,0.730539,0.809917
4,Word2vec,only_lemmatized_words,0.820937,0.820937,0.756126,0.820937
5,Word2vec,stopword + stemmed_words,0.776860,0.776860,0.702064,0.776860
6,Word2vec,stopword +lemmatized_words,0.801653,0.801653,0.722823,0.801653
7,Bag of words,no_preprocessing,0.785124,0.785124,0.681543,0.785124
8,Bag of words,only_stopword,0.746556,0.746556,0.609205,0.746556
9,Bag of words,only_stemmed_words,0.889807,0.889807,0.850263,0.889807
10,Bag of words,only_lemmatized_words,0.889807,0.889807,0.847692,0.889807


In [ ]:
dfoo.to_csv('/content/preprocessing_result_csv.csv', index= None)

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1,1))
X = vectorizer.fit_transform(data_train)
#create dataframe
cv_dataframe=pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
print(cv_dataframe)

       00  000  002  0025  003  0030  ...   ñn  ñnekoski  ñnnen  ñrnits  ñrvi  ñskyl
0     0.0  0.0  0.0   0.0  0.0   0.0  ...  0.0       0.0    0.0     0.0   0.0    0.0
1     0.0  0.0  0.0   0.0  0.0   0.0  ...  0.0       0.0    0.0     0.0   0.0    0.0
2     0.0  0.0  0.0   0.0  0.0   0.0  ...  0.0       0.0    0.0     0.0   0.0    0.0
3     0.0  0.0  0.0   0.0  0.0   0.0  ...  0.0       0.0    0.0     0.0   0.0    0.0
4     0.0  0.0  0.0   0.0  0.0   0.0  ...  0.0       0.0    0.0     0.0   0.0    0.0
...   ...  ...  ...   ...  ...   ...  ...  ...       ...    ...     ...   ...    ...
1806  0.0  0.0  0.0   0.0  0.0   0.0  ...  0.0       0.0    0.0     0.0   0.0    0.0
1807  0.0  0.0  0.0   0.0  0.0   0.0  ...  0.0       0.0    0.0     0.0   0.0    0.0
1808  0.0  0.0  0.0   0.0  0.0   0.0  ...  0.0       0.0    0.0     0.0   0.0    0.0
1809  0.0  0.0  0.0   0.0  0.0   0.0  ...  0.0       0.0    0.0     0.0   0.0    0.0
1810  0.0  0.0  0.0   0.0  0.0   0.0  ...  0.0       0.0    0.0  

In [ ]:
X_train_word_average.shape,X_test_word_average.shape

((1448, 300), (363, 300))

#logistic_regression

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(data_train, labels_train, test_size=0.20, random_state=42,stratify=labels_train)
#Classifier
clf=LogisticRegression(class_weight='balanced') 
clf_parameters = {'clf__solver':('newton-cg','lbfgs','liblinear'),}    
        
#Feature Extraction
pipeline = Pipeline([
('vect', CountVectorizer(token_pattern=r'\b\w+\b')),
('tfidf', TfidfTransformer(use_idf=True,smooth_idf=True)),     
('clf', clf),]) 
    
feature_parameters = {
"vect__stop_words": (None, 'english'),
'vect__analyzer':(stemmed_words, lemmatized_words),
'vect__min_df': (2,3),
'vect__ngram_range': ((1,1),(1, 2),(1,3),(2,3)),  # Unigrams, Bigrams or Trigrams
}

#Classificaion
parameters={**feature_parameters,**clf_parameters} 
grid = GridSearchCV(pipeline,parameters,scoring='f1_micro',cv=10)          
grid.fit(train_x,train_y)     
clf= grid.best_estimator_  
print('********* Best Set of Parameters ********* \n\n')
print(clf)

predicted = clf.predict(test_x)
predicted =list(predicted)
df0 = evaluation(train_x,train_y,test_y,test_x,predicted,'logistic_regression',df0)

********* Best Set of Parameters ********* 


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer=<function stemmed_words at 0x7f3925fa9170>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=3,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='\\b\\w+\\b', tok...
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                                    fit_intercept=True, int

In [ ]:
results = clf.predict(data_test)
results.shape

In [ ]:
len(results), len(data_test)

(453, 453)

In [ ]:
results

In [ ]:
df11 = pd.DataFrame()
df11['labels']= results
df11

,labels
0,positive
1,neutral
2,negative
3,neutral
4,positive
...,...
448,negative
449,neutral
450,neutral
451,positive


In [ ]:
df11.to_csv('test_labels.csv')

#naive_baies

In [ ]:

train_x, test_x, train_y, test_y = train_test_split(data_train, labels_train, test_size=0.20, random_state=42,stratify=labels_train)
#Classifier
clf=MultinomialNB(fit_prior=True, class_prior=None)

clf_parameters = {'clf__alpha':(0,1,2),}    
        
#Feature Extraction
pipeline = Pipeline([
('vect', CountVectorizer(token_pattern=r'\b\w+\b')),
('tfidf', TfidfTransformer(use_idf=True,smooth_idf=True)),     
('clf', clf),]) 
    
feature_parameters = {
"vect__stop_words": (None, 'english'),
'vect__analyzer':(stemmed_words, lemmatized_words),
'vect__min_df': (2,3),
'vect__ngram_range': ((1,1),(1, 2),(1,3),(2,3)),  # Unigrams, Bigrams or Trigrams
}

#Classificaion
parameters={**feature_parameters,**clf_parameters} 
grid = GridSearchCV(pipeline,parameters,scoring='f1_micro',cv=10)          
grid.fit(train_x,train_y)     
clf= grid.best_estimator_  
print('********* Best Set of Parameters ********* \n\n')
print(clf)

predicted = clf.predict(test_x)
predicted =list(predicted)
df0 = evaluation(train_x,train_y,test_y,test_x,predicted, 'naive_bayes',df0)

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/usr/local/lib/python3.7/dist-packages/sklearn/nai

********* Best Set of Parameters ********* 


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer=<function lemmatized_words at 0x7fe4370ac3b0>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=3,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='\\b\\w+\\b', tokenizer=None,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=0, class_prior=None, fit_prior=True))]

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


#SVM

In [ ]:
from sklearn.svm import SVC
train_x, test_x, train_y, test_y = train_test_split(data_train, labels_train, test_size=0.20, random_state=42,stratify=labels_train)
#Classifier
clf=SVC()

clf_parameters = {'clf__kernel':('linear', 'poly'),#, 'rbf', 'sigmoid'),
                  'clf__C' : (0,1),#,3,5,10),
                  }    
        
#Feature Extraction
pipeline = Pipeline([
('vect', CountVectorizer(token_pattern=r'\b\w+\b')),
('tfidf', TfidfTransformer(use_idf=True,smooth_idf=True)),     
('clf', clf),]) 
    
feature_parameters = {
#"vect__stop_words": (None, 'english'),
'vect__analyzer':(stemmed_words, lemmatized_words),
'vect__min_df': (2,3),
'vect__ngram_range': ((2,3),(1,1))#,(1, 2),(1,3)),  # Unigrams, Bigrams or Trigrams
}

#Classificaion
parameters={**feature_parameters,**clf_parameters} 
grid = GridSearchCV(pipeline,parameters,scoring='f1_micro',cv=10)          
grid.fit(train_x,train_y)     
clf= grid.best_estimator_  
print('********* Best Set of Parameters ********* \n\n')
print(clf)

predicted = clf.predict(test_x)
predicted =list(predicted)
df0 = evaluation(train_x,train_y,test_y,test_x,predicted,'svm',df0)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: b'C <= 0'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: b'C <= 0'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: b'C <= 0'

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: b'C <= 

********* Best Set of Parameters ********* 


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer=<function stemmed_words at 0x7fe4370ac170>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=2,
                                 ngram_range=(2, 3), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='\\b\\w+\\b', tok...
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 SVC(C=1, break_ties=False, cache_size=200, class_weight=None,
        

#random forest

In [ ]:
from sklearn.svm import SVC
train_x, test_x, train_y, test_y = train_test_split(data_train, labels_train, test_size=0.20, random_state=42,stratify=labels_train)
#Classifier
clf=RandomForestClassifier(random_state=42,n_estimators = 10)

clf_parameters = {'clf__criterion':('gini', 'entropy'),
                  'clf__max_depth' : (65,66,67,68,69,70),
                  'clf__max_features':('auto',0.75,0.8,0.4),
                  }    
        
#Feature Extraction
pipeline = Pipeline([
('vect', CountVectorizer(token_pattern=r'\b\w+\b')),
('tfidf', TfidfTransformer(use_idf=True,smooth_idf=True)),     
('clf', clf),]) 
    
feature_parameters = {
#"vect__stop_words": (None, 'english'),
'vect__analyzer':(stemmed_words, lemmatized_words),
'vect__min_df': (2,3),
'vect__ngram_range': ((2,3),(1,1))#,(1, 2),(1,3)),  # Unigrams, Bigrams or Trigrams
}

#Classificaion
parameters={**feature_parameters,**clf_parameters} 
grid = GridSearchCV(pipeline,parameters,scoring='f1_micro',cv=10)          
grid.fit(train_x,train_y)     
clf= grid.best_estimator_  
print('********* Best Set of Parameters ********* \n\n')
print(clf)

predicted = clf.predict(test_x)
predicted =list(predicted)
df0 = evaluation(train_x,train_y,test_y,test_x,predicted,'random_forest',df0)

********* Best Set of Parameters ********* 


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer=<function stemmed_words at 0x7fe4370ac170>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=2,
                                 ngram_range=(2, 3), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='\\b\\w+\\b', tok...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=65, max_features=0.4,
                                        max_leaf_nodes=None, max_samples=None,
                      

In [ ]:
df0.to_csv('/content/bagofwords_on_models_result_csv.csv')

In [ ]:
#

# word_to_vec

#logistic regression

In [ ]:

    
df_train = pd.DataFrame(data_train)
print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())

df_train[0] = df_train[0].apply(clean_text, args= (4,))
print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())
  
train_x, test_x, train_y, test_y = train_test_split(df_train, labels_train, test_size=0.2, random_state = 42)

test_tokenized = test_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values
train_tokenized = train_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values


X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

# clf=RandomForestClassifier(random_state=42,n_estimators = 10)

# clf_parameters = {'criterion':('gini', 'entropy'),
#                   'max_depth' : (65,66),#,67,68,69,70),
#                   'max_features':('auto',0.75),#,0.8,0.4),
#                   }    
        
clf=LogisticRegression(class_weight='balanced') 
clf_parameters = {'solver':('newton-cg','lbfgs','liblinear'),}
 
grid = GridSearchCV(clf, clf_parameters, scoring='f1_micro',cv=10)          
grid.fit(X_train_word_average,train_y)     
clf= grid.best_estimator_  
print('********* Best Set of Parameters ********* \n\n')
print(clf)

predicted = clf.predict(X_test_word_average)
predicted =list(predicted)
df0 = evaluation(X_train_word_average,train_y,test_y,X_test_word_average,predicted,'logistic_regression',df0)

words_per before 40627
words_per after 34802


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning:

********* Best Set of Parameters ********* 


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

 Total documents in the training set: 1448


 Total documents in the test set: 363


 Confusion Matrix 

[[ 39   4   9]
 [  9 200  10]
 [ 18  15  59]]

 Precision:0.8209366391184573

 Recall:0.8209366391184573

 Macro Averaged F1-Score :0.7561255351145952

 Mircro Averaged F1-Score:0.8209366391184573
                 model  precision    recall  micro_f1  macro_f1
0                 demo   0.000000  0.000000  0.000000  0.000000
1        random_forest   0.820937  0.820937  0.756126  0.820937
2  logistic_regression   0.820937  0.820937  0.756126  0.820937


In [ ]:

    
df_train = pd.DataFrame(data_train)
print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())

df_train[0] = df_train[0].apply(clean_text, args= (4,))
print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())
  
train_x, test_x, train_y, test_y = train_test_split(df_train, labels_train, test_size=0.2, random_state = 42)

test_tokenized = test_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values
train_tokenized = train_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values


X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

clf=RandomForestClassifier(random_state=42,n_estimators = 10)

clf_parameters = {'criterion':('gini', 'entropy'),
                  'max_depth' : (65,66),#,67,68,69,70),
                  'max_features':('auto',0.75),#,0.8,0.4),
                  }    
        
# clf=LogisticRegression(class_weight='balanced') 
# clf_parameters = {'solver':('newton-cg','lbfgs','liblinear'),}
# clf=MultinomialNB(fit_prior=True, class_prior=None)

# clf_parameters = {'alpha':(0,1,2),}    
        
 
grid = GridSearchCV(clf, clf_parameters, scoring='f1_micro',cv=10)          
grid.fit(X_train_word_average,train_y)     
clf= grid.best_estimator_  
print('********* Best Set of Parameters ********* \n\n')
print(clf)

predicted = clf.predict(X_test_word_average)
predicted =list(predicted)
df0 = evaluation(X_train_word_average,train_y,test_y,X_test_word_average,predicted,'SVM',df0)

words_per before 40627
words_per after 34802


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  app.launch_new_instance()


********* Best Set of Parameters ********* 


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=65, max_features=0.75,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

 Total documents in the training set: 1448


 Total documents in the test set: 363


 Confusion Matrix 

[[ 17  15  20]
 [  2 209   8]
 [ 15  28  49]]

 Precision:0.7575757575757576

 Recall:0.7575757575757576

 Macro Averaged F1-Score :0.6209013182453015

 Mircro Averaged F1-Score:0.7575757575757576
                 model  precision    recall  micro_f1  macro_f1
0                 demo   0.000000  0.000000 

In [ ]:

    
df_train = pd.DataFrame(data_train)
print('words_per before',df_train[0].apply(lambda x: len(x.split(' '))).sum())

df_train[0] = df_train[0].apply(clean_text, args= (4,))
print('words_per after',df_train[0].apply(lambda x: len(x.split(' '))).sum())
  
train_x, test_x, train_y, test_y = train_test_split(df_train, labels_train, test_size=0.2, random_state = 42)

test_tokenized = test_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values
train_tokenized = train_x.apply(lambda r: w2v_tokenize_text(r[0]), axis=1).values


X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

# clf=RandomForestClassifier(random_state=42,n_estimators = 10)

# clf_parameters = {'criterion':('gini', 'entropy'),
#                   'max_depth' : (65,66),#,67,68,69,70),
#                   'max_features':('auto',0.75),#,0.8,0.4),
#                   }    
        
# clf=LogisticRegression(class_weight='balanced') 
# clf_parameters = {'solver':('newton-cg','lbfgs','liblinear'),}
clf=RandomForestClassifier(random_state=42,n_estimators = 10)

clf_parameters = {'criterion':('gini', 'entropy'),
                  'max_depth' : (65,66,67,68,69,70),
                  'max_features':('auto',0.75,0.8,0.4),
                  }  
 
grid = GridSearchCV(clf, clf_parameters, scoring='f1_micro',cv=10)          
grid.fit(X_train_word_average,train_y)     
clf= grid.best_estimator_  
print('********* Best Set of Parameters ********* \n\n')
print(clf)

predicted = clf.predict(X_test_word_average)
predicted =list(predicted)
df0 = evaluation(X_train_word_average,train_y,test_y,X_test_word_average,predicted,'random_forest',df0)

words_per before 40627
words_per after 34802


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  app.launch_new_instance()


********* Best Set of Parameters ********* 


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=65, max_features=0.4,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

 Total documents in the training set: 1448


 Total documents in the test set: 363


 Confusion Matrix 

[[ 17  17  18]
 [  1 211   7]
 [ 12  29  51]]

 Precision:0.768595041322314

 Recall:0.768595041322314

 Macro Averaged F1-Score :0.6361105417776867

 Mircro Averaged F1-Score:0.768595041322314
                 model  precision    recall  micro_f1  macro_f1
0                 demo   0.000000  0.000000  0.0

In [ ]:
ss= df0.drop(0)
ss= ss.drop(1)

In [ ]:
ss.to_csv('\content\wor2vec on classifier.csv')

In [ ]:
ss= df1.drop(0)


In [ ]:
ss


,name,precision,recall,micro_f1,macro_f1
1,logistic_regression,0.983857,0.983857,0.964941,0.983857
2,naive_bayes,0.977578,0.977578,0.949102,0.977578
3,svm,0.982063,0.982063,0.959650,0.982063
